In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns

In [2]:
def change_rank(x):
    return (-1*(x-32))+1

In [27]:
def clean_stats_rb(run_stats, dic_def):
    run_stats = run_stats[(run_stats['Yds'] >= 10) & (run_stats['Att'] >= 3)]
    run_stats = run_stats.rename({'Unnamed: 7': 'Where'}, axis=1) 
    result = []
    points_home = []
    points_away = []
    for x in run_stats.Result:
        result.append(x[0])
        reg1 = re.findall(r'\d*[^-]', x)
        points_home.append(reg1[2])
        points_away.append(reg1[3])
    run_stats['Result'] = result
    run_stats['Points_Team'] = points_home
    run_stats['Points_Opp'] = points_away
    run_stats.Where.fillna('home', inplace=True)
    run_stats2 = run_stats.reset_index(drop=True)
    stadium = [] 
    for i, x in enumerate(run_stats2.Where):
        if x == 'home':
            stadium.append(1)
        else:
            stadium.append(0)
    run_stats2['Where'] = stadium
    dif_list = []
    for i, x in enumerate(run_stats2.Where):
        if x == 0:
            dif_list.append(dif_sta[run_stats2.Opp[i]]+32)
        else:
            dif_list.append(change_rank(dif_sta[run_stats2.Tm[i]]))
    run_stats2['Stad_Diff'] = dif_list
    run_stats3 = run_stats2.drop(['Lg', 'Tm', 'G#', 'Rk', 'Pos', 'Age', 'Att'], axis=1)
    run_stats3['Month'] = [x.month for x in run_stats3.Date]
    run_stats3['Month'] = run_stats3['Date'].map({9:'September', 10:'October', 11:'November', 12:'December'})
    run_stats3['Points_Team'] = pd.to_numeric(run_stats3.Points_Team, errors='coerce')
    run_stats3['Points_Opp'] = pd.to_numeric(run_stats3.Points_Opp, errors='coerce')
    run_stats3['Result'] = run_stats3.Result.map({'W':1, 'L':0, 'T':2})
    run_stats3['Opp'] = run_stats3['Opp'].map(dic_def)
    run_stats4 = run_stats3.set_index(['Date', 'Week', 'Player'])
    run_stats5 = pd.get_dummies(run_stats4)
    return run_stats5

In [28]:
def clean_stats_rb19_w4(df1, w1):
    w1['Date_October'] = [0 for x in range(len(w1))]
    w1['Date_November'] = [0 for x in range(len(w1))]
    w1['Date_December'] = [0 for x in range(len(w1))]
    w1['Day_Sat'] = [0 for x in range(len(w1))]
    w1 = w1[list(df1.columns)]
    run = pd.concat([df1, w1], sort=False)
    return run

In [29]:
dif_sta = {'TAM':6, 'GNB':21, 'NOR':26, 'KAN':31, 'BAL':18, 'WAS':15, 'NYJ':4, 'CIN':16, 'LAC':13, 'NWE':32, 'LAR':22,
           'MIN':29, 'IND':12, 'SEA':28, 'MIA':7, 'DEN':20, 'CAR':27, 'DAL':23, 'CHI':7, 'NYG':5, 'JAX':3, 'HOU':19,
           'OAK':9, 'PIT':30, 'ARI':17, 'ATL':24, 'BUF':10, 'CLE':1, 'PHI':25, 'DET':14, 'TEN':11, 'SFO':2}
dif_run_def = {'TAM':9, 'GNB':11, 'NOR':31, 'KAN':6, 'BAL':29, 'WAS':16, 'NYJ':7, 'CIN':4, 'LAC':24, 'NWE':22, 'LAR':10,
           'MIN':18, 'IND':25, 'SEA':20, 'MIA':2, 'DEN':12, 'CAR':21, 'DAL':28, 'CHI':32, 'NYG':13, 'JAX':14, 'HOU':30,
           'OAK':3, 'PIT':27, 'ARI':1, 'ATL':8, 'BUF':17, 'CLE':5, 'PHI':26, 'DET':23, 'TEN':15, 'SFO':19}

In [30]:
run_stats = pd.read_excel('../data/run/sportsref_rus1_18.xls')
for x in range(1,12):
    c = pd.read_excel(f'../data/run/sportsref_download ({x}).xls', skiprows=1)
    run_stats = pd.concat([run_stats, c], ignore_index = True, sort=False)

In [31]:
run1 = clean_stats_rb(run_stats, dif_run_def)

In [32]:
runw1 = pd.read_excel('../data/run/sportsref_download19_1(1).xls', skiprows=1)

In [33]:
run_statsw1 = clean_stats_rb(runw1, dif_run_def)

In [34]:
run_w1 = clean_stats_rb19_w4(run1, run_statsw1)

In [37]:
run_w1.TD.max()

4